In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import zipfile
import os

# Path to the uploaded zip file
zip_path = "animal_data.zip"
extract_path = "animal_data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Check the contents of the extracted directory
print(os.listdir(extract_path))


In [ ]:
import tensorflow as tf
print(tf.__version__)
!pip install Pillow



# **Animal Image Classification Model**

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input
import os
print(tf.__version__)


2.17.1


In [ ]:
# Define dataset directory
DATASET_DIR = "animal_data"

# Hyperparameters
BATCH_SIZE = 32
IMG_SIZE = (150, 150)
EPOCHS = 20

# Data Augmentation and Preprocessing
data_gen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values
    validation_split=0.2,  # 20% of data for validation
    rotation_range=20,  # Augmentation: rotate
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)


Load Training and Validation Data

In [ ]:
import PIL
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Custom Data Generator to handle bad images
def safe_image_generator(generator, directory, *args, **kwargs):
    """
    A wrapper to filter out invalid images during training and validation.
    """
    for batch in generator.flow_from_directory(directory, *args, **kwargs):
        try:
            # Ensure that all images are valid
            for img in batch[0]:
                PIL.Image.fromarray((img * 255).astype('uint8'))
            yield batch
        except Exception as e:
            print(f"Skipping invalid image batch: {e}")

# Data Augmentation and Preprocessing
data_gen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)

# Load Training and Validation Data Safely
train_data = safe_image_generator(
    data_gen,
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
)

validation_data = safe_image_generator(
    data_gen,
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=True,
)

CNN MODEL

In [ ]:
import os
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Input
from tensorflow.keras.callbacks import EarlyStopping

# Function to clean dataset
def clean_dataset(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    img.verify()
            except Exception as e:
                print(f"Removing corrupted file: {file_path} - {e}")
                os.remove(file_path)

# Define dataset parameters
DATASET_DIR = "animal_data"
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
EPOCHS = 20

# Clean the dataset
clean_dataset(DATASET_DIR)

# Data augmentation and preprocessing
data_gen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)

# Reload the generators after cleaning
train_data = data_gen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
)

validation_data = data_gen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=True,
)

# Get the number of classes dynamically
temp_iterator = data_gen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=False,
)
num_classes = len(temp_iterator.class_indices)
print(f"Number of classes: {num_classes}")

# Define the CNN model
model = models.Sequential([
    Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax'),
])

model.summary()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Add early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
try:
    history = model.fit(
        train_data,
        epochs=EPOCHS,
        validation_data=validation_data,
        callbacks=[early_stopping],
    )
except Exception as e:
    print(f"Training failed: {e}")

# Evaluate the model
try:
    eval_loss, eval_accuracy = model.evaluate(validation_data)
    print(f"Validation Accuracy: {eval_accuracy:.2f}")
except Exception as e:
    print(f"Evaluation failed: {e}")

# Save the model in the recommended format
model.save("animal_classification_model.keras")
print("Model saved as animal_classification_model.keras")



Removing corrupted file: animal_data/__MACOSX/._animal_data - cannot identify image file '/content/animal_data/__MACOSX/._animal_data'
Removing corrupted file: animal_data/__MACOSX/animal_data/._Giraffe - cannot identify image file '/content/animal_data/__MACOSX/animal_data/._Giraffe'
Removing corrupted file: animal_data/__MACOSX/animal_data/._Kangaroo - cannot identify image file '/content/animal_data/__MACOSX/animal_data/._Kangaroo'
Removing corrupted file: animal_data/__MACOSX/animal_data/._Panda - cannot identify image file '/content/animal_data/__MACOSX/animal_data/._Panda'
Removing corrupted file: animal_data/__MACOSX/animal_data/._Deer - cannot identify image file '/content/animal_data/__MACOSX/animal_data/._Deer'
Removing corrupted file: animal_data/__MACOSX/animal_data/._Elephant - cannot identify image file '/content/animal_data/__MACOSX/animal_data/._Elephant'
Removing corrupted file: animal_data/__MACOSX/animal_data/._Bird - cannot identify image file '/content/animal_data/

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_24 (Conv2D)                   │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_24 (MaxPooling2D)      │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_25 (Conv2D)                   │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_25 (MaxPooling2D)      │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_26 (Conv2D)                   │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_26 (MaxPooling2D)      │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_8 (Flatten)                  │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 128)                 │       4,735,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,828,610 (18.42 MB)

 Trainable params: 4,828,610 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


49/49 ━━━━━━━━━━━━━━━━━━━━ 98s 2s/step - accuracy: 1.0000 - loss: 0.0630 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 140s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/20
25/49 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00

KeyboardInterrupt: 

Evaluation And Prediction

In [ ]:
# Evaluate the Model
eval_loss, eval_accuracy = model.evaluate(validation_data)
print(f"Validation Accuracy: {eval_accuracy:.2f}")

# Save the Model
model.save("animal_classification_model.h5")
print("Model saved as animal_classification_model.h5")

# Predict on Validation Data
import numpy as np
from matplotlib import pyplot as plt

# Get a batch of validation data
validation_batch = next(iter(validation_data))
images, labels = validation_batch

predictions = model.predict(images)

# Display predictions
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(images[i])
    pred_label = np.argmax(predictions[i])
    true_label = np.argmax(labels[i])
    plt.title(f"Pred: {pred_label}, True: {true_label}")
    plt.axis('off')
plt.tight_layout()
plt.show()

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7bcc89a483b0>